### 决策树案例

In [1]:
# 1. build dataset
from math import log
import operator

def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    #change to discrete values
    return dataSet, labels

In [2]:
# 2. calc entory
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: # the the number of unique elements and their occurance
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): 
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * log(prob,2) # log base 2
    return shannonEnt

In [3]:
# 3.split data
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[: axis]     # chop out axis used for splitting
            reducedFeatVec.extend(featVec[axis+1: ])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [5]:
# 4.choose best feature
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1      # the last column is used for the labels
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):        # iterate over all the features
        featList = [example[i] for example in dataSet]  # create a list of all the examples of this feature
        uniqueVals = set(featList)       # get a set of unique values
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)     
        infoGain = baseEntropy - newEntropy     # calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain):       # compare this to the best gain so far
            bestInfoGain = infoGain         # if better than current best, set to best
            bestFeature = i
    return bestFeature                      # returns an integer

In [7]:
# 5. major count label
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): 
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [8]:
# 6. creat tree
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): 
        return classList[0]   # stop splitting when all of the classes are equal
    if len(dataSet[0]) == 1:  # stop splitting when there are no more features in dataSet
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]  # copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree    

In [9]:
# 7. classify by builded tree
def classify(inputTree, featLabels, testVec):
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): 
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: 
        classLabel = valueOfFeat
    return classLabel

In [10]:
# 8. save tree
def storeTree(inputTree,filename):
    import pickle
    fw = open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()

In [12]:
# 9. reload tree
def grabTree(filename):
    import pickle
    fr = open(filename)
    return pickle.load(fr)

### 测试

In [15]:
# 1. 查看数据
data, label = createDataSet()
print(data, label)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']] ['no surfacing', 'flippers']


In [21]:
# 2. 计算当前数据的熵
base_entroy = calcShannonEnt(data)
print(base_entroy)

0.9709505944546686


In [17]:
# 3, 测试按照指定特征划分数据
splitDataSet(data, 0, 1)  # 0:表示第1个特征， 1:表示特征值为1

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [18]:
splitDataSet(data, 0, 0)

[[1, 'no'], [1, 'no']]

In [19]:
chooseBestFeatureToSplit(data)  # res = 0 表示 第 0 个特征最好的

0

In [22]:
data1 = [[1, 'yes'], [1, 'yes'], [0, 'no']]
data0 = [[1, 'no'], [1, 'no']]
entroy1 = calcShannonEnt(data1)
entroy0 = calcShannonEnt(data0)
entroy1_diff = base_entroy - entroy1
entroy0_diff = base_entroy - entroy0
print(entroy1_diff, entroy0_diff)

0.052654760400179024 0.9709505944546686


In [25]:
tree = createTree(data, label)
print(tree)

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
